# Performance Analysis of Natural Language Processing Techniques in Radiology Report Classification - Brain MRI and Head CT dataset

- Author: Eric Yang
- Created September 21st, 2021

## Imports

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import loguniform
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

from xgboost import XGBClassifier, plot_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.utils import resample, shuffle
from nltk import word_tokenize
from nltk.sentiment.util import mark_negation

## Data Preprocessing and Partitioning

In [ ]:
# read in brain MRI and head CT data
mri_data = pd.read_csv('../Data/Brain_MRI_head_CT/sample1000_brainMRIs_2017to2020_Min_FINAL.csv').dropna(subset = ['IMPRESSION'])
ct_data = pd.read_csv('../Data/Brain_MRI_head_CT/sample1000_headCTs_2017to2020_FD_FINAL.csv').dropna(subset = ['IMPRESSION'])
head_data = pd.concat([mri_data, ct_data])

# remove name column
head_data.drop('NAME', axis = 1, inplace = True)

# for each 'REPORT', filter for sentences that contain desired expression
def excerpt_fun(data, column, expression_list):
    # data is df
    # column is either 'REPORT' or 'IMPRESSION'
    # expression_list is a list of regex expressions of interest
    excerpts = []
    for i in tqdm(range(len(data))):
        text = data.iloc[i][column]
        matches = []
        for exp in expression_list:
            matches += re.findall(exp, text, flags=re.IGNORECASE) 
        excerpt = ' '.join([a[0] for a in matches])
        excerpts.append(excerpt)
    return(excerpts)

# filter for sentences of interest
head_data['impression_excerpts'] = excerpt_fun(head_data,'IMPRESSION', [r"([^.]*?(infarct|ischem)[^.]*\.)"])

# train test split on MRN column
all_MRNs = list(set(head_data['MRN']))
np.random.seed(314)
np.random.shuffle(all_MRNs)
training_MRNs = all_MRNs[0:int(len(all_MRNs)*0.8)]
testing_MRNs = all_MRNs[int(len(all_MRNs)*0.8):]
training_data = head_data[head_data['MRN'].isin(training_MRNs)]
testing_data = head_data[head_data['MRN'].isin(testing_MRNs)]

# get training and testing corpus, labels
all_corpus = list(head_data['impression_excerpts'])
training_corpus = list(training_data['impression_excerpts'])
training_infarct_labels = np.array(list(training_data['ACUTE_INFARCT']))

testing_corpus = list(testing_data['impression_excerpts'])
testing_infarct_labels = np.array(list(testing_data['ACUTE_INFARCT']))

## Define evaluation metrics and bootstrapping workflow

In [ ]:
# define evaluation functions
def specificity(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    spec = tn / (tn+fp)
    return spec

# Testing with bootstrap 95%
def test_bootstrap(X_test, test_labels, clf):
    '''
    Testing of algorithm with generation of bootstrap 95% CI
    Output:
    tuples with performance metrics and boostrap 95% CI (5%, 50%, 95%tiles)
    '''
    accuracy_scores = []
    cohen_kappa_scores = []
    precision_scores = []
    recall_scores = []
    specificity_scores = []
    f1_scores = []
    for i in tqdm(range(1000)):
        testing_sample, testing_labels = resample(X_test, test_labels, replace=True)
        predicted = clf.predict(testing_sample)
        accuracy_scores.append(accuracy_score(testing_labels, predicted))
        precision_scores.append(precision_score(testing_labels, predicted))
        recall_scores.append(recall_score(testing_labels, predicted))
        specificity_scores.append(specificity(testing_labels, predicted))
        f1_scores.append(f1_score(testing_labels, predicted))
    accuracy_scores = sorted(accuracy_scores, reverse = False)
    precision_scores = sorted(precision_scores, reverse = False)
    recall_scores = sorted(recall_scores, reverse = False)
    specificity_scores = sorted(specificity_scores, reverse = False)
    f1_scores = sorted(f1_scores, reverse = False)
    accuracy_CI = (accuracy_scores[24], accuracy_scores[499], accuracy_scores[974])
    precision_CI = (precision_scores[24], precision_scores[499], precision_scores[974])
    recall_CI = (recall_scores[24], recall_scores[499], recall_scores[974])
    specificity_CI = (specificity_scores[24], specificity_scores[499], specificity_scores[974])
    f1_CI = (f1_scores[24], f1_scores[499], f1_scores[974])
    results = {'accuracy_CI':accuracy_CI,
               'precision_CI':precision_CI,
               'recall_CI':recall_CI,
               'specificity_CI':specificity_CI,
               'f1_CI':f1_CI
               }
    return results

## Bag of words with NLTK

In [ ]:
# Bag of words with NLTK
vectorizer = CountVectorizer(analyzer = 'word', 
                             tokenizer = lambda text: mark_negation(word_tokenize(text)),  
                             ngram_range = (1,3), 
                             min_df = 0.05
                             )
X_train_bow = vectorizer.fit_transform(training_corpus).toarray()
vectorizer_features = vectorizer.get_feature_names()
X_test_bow = vectorizer.transform(testing_corpus).toarray()

## XGBoost with bag of words

In [ ]:
# XGBoost training using BOW for acute/subacute ischemic infarct prediction
param_dist = {'colsample_bytree':uniform(0.01,1),
              'eta':uniform(0,1),
              "max_depth": sp_randint(3,11),
              "min_child_weight": sp_randint(1, 11)}
XGBmodel_bow_infarct = XGBClassifier()
random_search = RandomizedSearchCV(XGBmodel_bow_infarct, param_distributions=param_dist, scoring='f1',random_state=314)
search = random_search.fit(X_train_bow, training_infarct_labels)
XGBmodel_bow_infarct = search.best_estimator_

# evaluate performance on acute/subacute ischemic infarct, BOW XGBoost model
results_infarct_xgb_bow = test_bootstrap(X_test_bow, testing_infarct_labels, XGBmodel_bow_infarct)
print(results_infarct_xgb_bow)

# look at important features of BOW XGBoost model on acute/subacute ischemic infarct prediction
# importance = how many times each feature is split on
XGBmodel_bow_infarct.get_booster().feature_names = vectorizer_features
plot_importance(XGBmodel_bow_infarct, max_num_features=10, title = 'Feature importance of BOW XGBoost model trained for acute/subacute ischemic infarct prediction')
plt.show() 

## Random forest with bag of words

In [ ]:
# Random Forest training using BOW for acute/subacute ischemic infarct prediction
param_dist = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)],
              'max_features': ['auto', 'sqrt'],
              "max_depth": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
              "min_samples_leaf": sp_randint(1, 11),
              'min_samples_split': [2, 5, 10]}
RFmodel_bow_infarct = RandomForestClassifier()
random_search = RandomizedSearchCV(RFmodel_bow_infarct, param_distributions=param_dist, scoring='f1',random_state=314)
search = random_search.fit(X_train_bow, training_infarct_labels)
RFmodel_bow_infarct = search.best_estimator_

# evaluate performance on acute/subacute ischemic infarct, BOW RF model
results_infarct_rf_bow = test_bootstrap(X_test_bow, testing_infarct_labels, RFmodel_bow_infarct)
print(results_infarct_rf_bow)

## Logistic regression with bag of words

In [ ]:
# Logistic regression training using BOW for acute/subacute ischemic infarct prediction
param_dist = {'solver': ['newton-cg', 'lbfgs', 'liblinear'],
              'penalty': ['none', 'l1', 'l2', 'elasticnet'],
              'C': loguniform(1e-5, 100)}
LOGmodel_bow_infarct = LogisticRegression()
random_search = RandomizedSearchCV(LOGmodel_bow_infarct, param_distributions=param_dist, scoring='f1',random_state=315)
search = random_search.fit(X_train_bow, training_infarct_labels)
LOGmodel_bow_infarct = search.best_estimator_

# evaluate performance on acute/subacute ischemic infarct, BOW logistic regression model
results_infarct_log_bow = test_bootstrap(X_test_bow, testing_infarct_labels, LOGmodel_bow_infarct)
print(results_infarct_log_bow)

## Load external validation dataset

In [ ]:
# read in brain MRI and head CT data
ext_head_data = pd.read_csv('../Data/Brain_MRI_Head_CT/CT_MR_data_concatenated.csv', index_col = 0).dropna(subset = ['IMPRESSION'])

# remove name column
ext_head_data.drop('Name', axis = 1, inplace = True)

# filter for sentences of interest
ext_head_data['impression_excerpts'] = excerpt_fun(ext_head_data,'IMPRESSION', [r"([^.]*?(infarct|ischem)[^.]*\.)"])

# get training and testing corpus, labels
ext_all_corpus = list(ext_head_data['impression_excerpts'])
ext_infarct_labels = np.array(list(ext_head_data['ACUTE_INFARCT']))

In [ ]:
# explore data
print("Total number of patients:", len(set(ext_head_data['Mrn'])))
print("Total number of brain MRIs and head CTs:", len(ext_head_data))
print("Total number of acute/subacute ischemic infarct cases:", len(ext_head_data[ext_head_data['ACUTE_INFARCT'] == 1]))

## Perform subsampling experiments
- This is essentially the first half of the notebook on loop

In [ ]:
# evaluate each NLP model for each proportion

for i, (prop, tag) in enumerate(zip(proportions, file_tags)):
    # evaluate performance on acute/subacute ischemic infarct, BOW XGBoost model, ext dataset
    results_infarct_xgb_bow = test_bootstrap(X_test_bow, ext_infarct_labels, xgb_bow_models[i])
    ext_xgb_bow_results.append(results_infarct_xgb_bow)
    
    # evaluate performance on acute/subacute ischemic infarct, BOW RF model, ext dataset
    results_infarct_rf_bow = test_bootstrap(X_test_bow, ext_infarct_labels, rf_bow_models[i])
    ext_rf_bow_results.append(results_infarct_rf_bow)
    
    # evaluate performance on acute/subacute ischemic infarct, BOW log reg model, ext dataset
    results_infarct_log_bow = test_bootstrap(X_test_bow, ext_infarct_labels, log_bow_models[i])
    ext_log_bow_results.append(results_infarct_log_bow)

In [ ]:
# define proportions
proportions = np.array([0.025, 0.10, 0.25, 0.50, 0.75, 1.00])
file_tags = np.array(['025', '10', '25', '50', '75', ''])

# loop through NLP workflow for each proportion
xgb_bow_sampling_results = []
rf_bow_sampling_results = []
log_bow_sampling_results = []
ext_xgb_bow_results = []
ext_rf_bow_results = []
ext_log_bow_results = []
xgb_clin_ent_sampling_results = []
xgb_clin_emb_sampling_results = []
for prop, tag in zip(proportions, file_tags):
    n_samp = round(len(training_corpus)*prop)
    np.random.seed(314)
    sample_indices = np.random.choice(range(len(training_corpus)), n_samp, replace = False)
    sub_training_data = training_data.iloc[sample_indices,:]
    sub_training_data.to_csv('../Data/Brain_MRI_Head_CT/training_data' + tag + '.csv')
    sub_training_corpus = sub_training_data['impression_excerpts']
    sub_training_labels = sub_training_data['ACUTE_INFARCT']
    
    # XGB BOW
    param_dist = {'colsample_bytree':uniform(0.01,1),
              'eta':uniform(0,1),
              "max_depth": sp_randint(3,11),
              "min_child_weight": sp_randint(1, 11)}
    X_train_bow = vectorizer.fit_transform(sub_training_corpus).toarray()
    vectorizer_features = vectorizer.get_feature_names()
    X_test_bow = vectorizer.transform(testing_corpus).toarray()
    XGBmodel_bow_infarct = XGBClassifier()
    random_search = RandomizedSearchCV(XGBmodel_bow_infarct, param_distributions=param_dist, scoring='f1',random_state=314)
    search = random_search.fit(X_train_bow, sub_training_labels)
    XGBmodel_bow_infarct = search.best_estimator_
    # evaluate performance on acute/subacute ischemic infarct, BOW XGBoost model
    results_infarct_xgb_bow = test_bootstrap(X_test_bow, testing_infarct_labels, XGBmodel_bow_infarct)
    xgb_bow_sampling_results.append(results_infarct_xgb_bow)
    ext_X_test_bow = vectorizer.transform(ext_all_corpus).toarray()
    ext_results_infarct_xgb_bow = test_bootstrap(ext_X_test_bow, ext_infarct_labels, XGBmodel_bow_infarct)
    ext_xgb_bow_results.append(ext_results_infarct_xgb_bow)
    
    # Random Forest training using BOW for acute/subacute ischemic infarct prediction
    param_dist = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)],
                  'max_features': ['auto', 'sqrt'],
                  "max_depth": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
                  "min_samples_leaf": sp_randint(1, 11),
                  'min_samples_split': [2, 5, 10]}
    RFmodel_bow_infarct = RandomForestClassifier()
    random_search = RandomizedSearchCV(RFmodel_bow_infarct, param_distributions=param_dist, scoring='f1',random_state=314)
    search = random_search.fit(X_train_bow, sub_training_labels)
    RFmodel_bow_infarct = search.best_estimator_
    # evaluate performance on acute/subacute ischemic infarct, BOW RF model
    results_infarct_rf_bow = test_bootstrap(X_test_bow, testing_infarct_labels, RFmodel_bow_infarct)
    rf_bow_sampling_results.append(results_infarct_rf_bow)
    ext_results_infarct_rf_bow = test_bootstrap(ext_X_test_bow, ext_infarct_labels, RFmodel_bow_infarct)
    ext_rf_bow_results.append(ext_results_infarct_rf_bow)
    
    # Logistic regression training using BOW for acute/subacute ischemic infarct prediction
    param_dist = {'solver': ['newton-cg', 'lbfgs', 'liblinear'],
                  'penalty': ['none', 'l1', 'l2', 'elasticnet'],
                  'C': loguniform(1e-5, 100)}
    LOGmodel_bow_infarct = LogisticRegression()
    random_search = RandomizedSearchCV(LOGmodel_bow_infarct, param_distributions=param_dist, scoring='f1',random_state=315)
    search = random_search.fit(X_train_bow, sub_training_labels)
    LOGmodel_bow_infarct = search.best_estimator_
    # evaluate performance on acute/subacute ischemic infarct, BOW logistic regression model
    results_infarct_log_bow = test_bootstrap(X_test_bow, testing_infarct_labels, LOGmodel_bow_infarct)
    log_bow_sampling_results.append(results_infarct_log_bow)
    ext_results_infarct_log_bow = test_bootstrap(ext_X_test_bow, ext_infarct_labels, LOGmodel_bow_infarct)
    ext_log_bow_results.append(ext_results_infarct_log_bow)

## Visualize results

In [ ]:
# plot results
condition = ['Acute/subacute Ischemic Infarct']
all_results_infarct = [xgb_bow_sampling_results, rf_bow_sampling_results, log_bow_sampling_results] 
all_results = [all_results_infarct]
method_names = ['XGBoost w/ bag of words', 'Random Forest w/ bag of words', 'Logistic Regression w/ bag of words']
metrics = ['Accuracy', 'Precision', 'Recall', 'Specificity', 'F1 Score']
sample_size = np.around(len(training_corpus)*proportions)
for cond, res in zip(condition, all_results): # for each condition, we only have 1 here
    for metric_i, metric in enumerate(metrics): # for each metric
        for method_results, method_name in zip(res, method_names): # for each NLP method
            means = []
            lower = []
            upper = []
            for prop in range(len(sample_size)):
                lower.append(list(method_results[prop].values())[metric_i][0])
                means.append(list(method_results[prop].values())[metric_i][1])
                upper.append(list(method_results[prop].values())[metric_i][2])
            plt.errorbar(sample_size, means, 
            yerr = [(np.array(means)-np.array(lower)).tolist(),(np.array(upper)-np.array(means)).tolist()], 
            fmt = '-o',
            label = method_name)
        plt.ylim([0,1])
        plt.xlabel('Number of Training Samples',size=13)
        plt.ylabel(metric,size=13)
        plt.title(cond + ' Classification',size=13)
        plt.legend(loc="lower center",ncol=2, bbox_to_anchor=(0.5, -0.4))
        plt.show()

In [ ]:
# plot results
condition = ['Acute/subacute Ischemic Infarct']
ext_results = [ext_xgb_bow_results, ext_rf_bow_results, ext_log_bow_results]
all_results = [ext_results]
method_names = ['XGBoost w/ bag of words', 'Random Forest w/ bag of words', 'Logistic Regression w/ bag of words']
metrics = ['Accuracy', 'Precision', 'Recall', 'Specificity', 'F1 Score']
sample_size = np.around(len(training_corpus)*proportions)
for cond, res in zip(condition, all_results): # for each condition, we only have 1 here
    for metric_i, metric in enumerate(metrics): # for each metric
        for method_results, method_name in zip(res, method_names): # for each NLP method
            means = []
            lower = []
            upper = []
            for prop in range(len(sample_size)):
                lower.append(list(method_results[prop].values())[metric_i][0])
                means.append(list(method_results[prop].values())[metric_i][1])
                upper.append(list(method_results[prop].values())[metric_i][2])
            plt.errorbar(sample_size, means, 
            yerr = [(np.array(means)-np.array(lower)).tolist(),(np.array(upper)-np.array(means)).tolist()], 
            fmt = '-o',
            label = method_name)
        plt.ylim([0,1])
        plt.xlabel('Number of Training Samples',size=13)
        plt.ylabel(metric,size=13)
        plt.title(cond + ' Classification',size=13)
        plt.legend(loc="lower center",ncol=2, bbox_to_anchor=(0.5, -0.4))
        plt.show()